**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install hyperopt
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from hyperopt import hp, tpe
from hyperopt.pyll.base import scope
from hyperopt.fmin import fmin
from hyperopt import space_eval
from sklearn.model_selection import cross_validate

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/u8u7vcwy3sosbar/titanic.zip?dl=1",
                            directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Bayesovská optimalizácia hyperparametrov

Ako pomocou balíčka `scikit-learn` zostaviť a natrénovať jednoduchý model už vieme. Vieme tiež, že modely majú hyperparametre: t.j. parametre, ktoré sa neladia počas učenia, ale sa musia nejakým spôsobom nastaviť dopredu alebo sa, v ideálnom prípade, vyberú pomocou niektorej metódy na optimalizáciu hyperparametrov.

Tréning modelu je však často veľmi výpočtovo náročný a musí sa opakovať pre mnoho rôznych množín hyperparametrov kým sa nájde optimálna konfigurácia. Práve preto sa na ladenie hyperparametrov často používa bayesovská optimalizácia – ako sme už povedali, je to prístup, ktorého cieľom je nájsť optimum účelovej funkcie pri čo najmenšom počte skutočných dopytov na jej hodnotu.

V tomto notebooku ukážeme praktický prístup ku bayesovskej optimalizácii hyperparametrov pomocou populárneho balíčka `hyperopt`.

### Načítanie dát a predspracovanie

Ako obvykle, začneme načítaním a predspracovaním dát. Budeme pracovať so známou dátovou množinou [Titanic](https://www.kaggle.com/c/titanic). Keďže v našom prípade nebude potrebné proces načítania a predspracovania dát prejsť podrobne, kód nasledujúcej bunky je skrytý.



In [ ]:
#@title -- Loading and preprocessing: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

# we split the columns into categorical and numeric inputs and the output
categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']
output = ["Survived"]

# we create our preprocessing pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder(categories='auto')),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# we fit the pipeline on the train set and then apply it to both train and test
X_train = input_preproc.fit_transform(df_train[categorical_inputs + numeric_inputs])
Y_train = df_train[output]

X_test = input_preproc.transform(df_test[categorical_inputs + numeric_inputs])
Y_test = df_test[output]

### Bayesovská optimalizácia

Prvá vec, ktorú bude potrebné urobiť pre samotnou aplikáciou bayesovskej optimalizácie, je samozrejme definovať účelovú funkciu, ktorú bude optimalizácia minimalizovať.

Keďže naším cieľom bude nájsť hyperparametre, pri ktorých náš model dosiahne najlepšie výsledky, vstupným argumentom budú tieto hyperparametre. S ich pomocou skonštruujeme model (rozhodovací strom na báze triedy `DecisionTreeClassifier`).

Správnosť modelu vyhodnotíme pomocou $k$-násobnej krížovej validácie. (Cross-validation. Tréningové dáta rozdelíme na $k$ častí, pričom zakaždým jednu časť použijeme na testovanie a zvyšné na trénovanie. Keď takto otestujeme model na všetkých kombináciách tréningových a testovacích množín, výslednú správnosť určíme ako priemer správností z jednotlivých behov.)



In [ ]:
def objective(params):
    model = DecisionTreeClassifier(**params)
    
    score = cross_validate(model, X_train, Y_train,
                           scoring='f1_macro',
                           cv=10, n_jobs=10)['test_score'].mean()
    print("Score {:.3f} params {}".format(score, params))

    # minus because we want the score to be as high as
    # possible, but the objective function is to be minimized
    return -score

Ďalej potrebujeme nakonfigurovať prehľadávaný priestor: t.j. špecifikovať, akými hyperparametrami disponuje naša metóda a určiť, aké hodnoty môžu nadobúdať. Začnime teda tým, že si zobrazíme dokumentáciu ku triede `DecisionTreeClassifier`.



In [ ]:
?DecisionTreeClassifier

---
### Úloha 1: Konfigurácia prehľadávaného priestoru

**V nasledujúcej bunke definujte prehľadávaný priestor `space` pre hyperparametre rozhodovacieho stromu.** 

---
Priestor sa definuje slovníkom v nasledujúcom tvare:

```
space = {
    # kategorická premenná:
    'cat_var': hp.choice("cat_var", ["opt1", "opt2", "opt3"]),

    # celočíselná premenná z rovnomerného rozdelenia:
    'int_var': scope.int(hp.quniform("int_var", 1, 15, 1)),

    # reálnočíselná premenná z rovnomerného rozdelenia:
    'float_var': hp.uniform('float_var', 0.2, 1.0),
}
```
Ďalšie možnosti a podrobnejšia dokumentácia ku definovaniu parametrických priestorov sa dajú nájsť na [wiki balíčka hyperopt](https://github.com/hyperopt/hyperopt/wiki/FMin#21-parameter-expressions).



In [ ]:
space = {
    
    
    # ---
    
    
}

### Spustenie optimalizácie

Ďalej už môžeme spustiť samotnú optimalizáciu. Špecifikujeme pritom účelovú funkciu, prehľadávaný priestor, maximálny počet vyhodnotení účelovej funkcie a algoritmus. My používame algoritmus `tpe`, tzv. Tree-structured Parzen Estimator. Táto metóda dosahuje pri vysokorozmerných priestoroch lepšie výsledky než gaussovské procesy, základné ciele však sleduje rovnaké.



In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100
        )

Funkcia `fmin` navráti najlepšie nájdené riešenie. Následne ho dekódujeme pomocou funkcie `space_eval`, čím získame reprezentáciu, ktorú je už možné priamo použiť pri vytváraní nášho modelu.



In [ ]:
best_params = space_eval(space, best)

### Tréning modelu s najlepšími hyperparametrami

Keď sme identifikovali najlepšie parametre, použijeme ich teraz, aby sme natrénovali nový model: tento raz už s použitím celej tréningovej množiny.



In [ ]:
model = DecisionTreeClassifier(**best_params)
model.fit(X_train, Y_train)

### Testovanie

Na záver si model otestujeme na testovacích dátach. Zobrazíme si maticu zámen a naše štandardné metriky.



In [ ]:
y_test = model.predict(X_test)

In [ ]:
cm = pd.crosstab(Y_test.values.reshape(-1), y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm)

In [ ]:
print("Accuracy = {}".format(accuracy_score(Y_test, y_test)))
print("Precision = {}".format(precision_score(Y_test, y_test)))
print("Recall = {}".format(recall_score(Y_test, y_test)))

Úspešnosť predikcie by mala byť lepšia než pri predvolených hyperparametroch (aspoň v priemer – kvôli stochasticite je ťažké povedať niečo zmysluplné o jednom konkrétnom behu). Môžeme si to skúsiť aj overiť.



In [ ]:
def_model = DecisionTreeClassifier()
def_model.fit(X_train, Y_train)
y_test = def_model.predict(X_test)

print("Accuracy = {}".format(accuracy_score(Y_test, y_test)))
print("Precision = {}".format(precision_score(Y_test, y_test)))
print("Recall = {}".format(recall_score(Y_test, y_test)))

---
### Úloha 2: Optimalizácia hyperparametrov pre XGBoost

**Vyskúšajte teraz celý postup zopakovať s inou klasifikačnou metódou: s metódou XGBClassifier z balíčka `xgboost`. Predefinovať bude potrebné najmä metódu `objective`, aby používala nový model a prehľadávaný priestor `space` tak aby zodpovedal hyperparametrom novej metódy.** 

---


In [ ]:
from xgboost import XGBClassifier